In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<p style="font-family: Arial; font-size:3.75vw;color:purple; font-style:bold"><br>
pandas Exercise Notebook
</p><br>

# Exercise Notebook Instructions

### 1. Important: Only modify the cells which instruct you to modify them - leave "do not modify" cells alone.  

The code which tests your responses assumes you have run the startup/read-only code exactly.

### 2. Work through the notebook in order.

Some of the steps depend on previous, so you'll want to move through the notebook in order.

### 3. It is okay to use libraries.

You may find some questions are fairly straightforward to answer using built-in library functions.  That's totally okay - part of the point of these exercises is to familiarize you with the commonly used functions.

### 4. Seek help if stuck

If you get stuck, don't worry!  You can either review the videos/notebooks from this week, ask in the course forums, or look to the solutions for the correct answer.  BUT, be careful about looking to the solutions too quickly.  Struggling to get the right answer is an important part of the learning process.

In [ ]:
import pandas as pd
import numpy as np
import os.path

In this exercise notebook you will have the opportunity to load the MovieLens database and perform additional analysis.

First let's load the data into a Pandas Dataframe:

In [ ]:
# DO NOT MODIFY

# set here the relative path to the movielens folder
MOVIELENS="../input/movielens/ml-20m/"

movies = pd.read_csv(os.path.join(MOVIELENS, 'movies.csv'), sep=',')
ratings = pd.read_csv(os.path.join(MOVIELENS, 'ratings.csv'), sep=',')

In [ ]:
movies.head()

In [ ]:
ratings.head()

In [ ]:
import string 

years = [] #list of years as ints
error = 0

for e in range(len(movies.title)):
    try:
        years.append(int(movies.title[e].split()[-1].strip("()" + string.ascii_letters)))
    except:
        years.append(0)
        error += 1
        
print("Error percentage: " + str(round(error/len(movies.title) * 100, 3)) + " %")
movies["year"] = years

movies.head()

## Exercise 1: Find the Minimum Rating

Let's start by computing the minimum rating.

In the next cell, define the `min_rating` variable to be the minimum rating across all of the DataFrame:

In [ ]:
min_rating = None
### BEGIN SOLUTION
min_rating = ratings['rating'].min()
### END SOLUTION

In [ ]:
# DO NOT MODIFY

assert isinstance(min_rating, float), "Try again, make sure you are taking the min of just 1 column"
assert abs(min_rating - 0.5) < .01, "Try again, the minimum should be 0.5"

## Exercise 2: Find the Mean Rating of a Movie

The movie **Toy Story** has `movieId` 1. Find out the mean rating for Toy Story. For this exercise you just need to use the `ratings` DataFrame:


In [ ]:
toy_story_rating = None
### BEGIN SOLUTION
toy_story_rating = ratings['rating'][ratings['movieId'] == 1].mean()
print(toy_story_rating)
### END SOLUTION

In [ ]:
assert abs(toy_story_rating - 3.89) < 0.04, "Try again, select only the rows where the movieId is equal to 1"

## Exercise 3: Find the Most Common Rating

Next, find which rating is the most common.

In [ ]:
rating_counts_mode = None
### BEGIN SOLUTION
rating_counts_mode = ratings.rating.mode()[0]
print(rating_counts_mode)
### END SOLUTION

In [ ]:
# DO NOT MODIFY

assert rating_counts_mode == 4.0, "Print out rating_counts and try to understand what is wrong"

## Exercise 4: Usage of the Index in pandas

In `numpy` the way to point to a specific entry in an array is by using its integer position. In `pandas` you can do the same with `iloc`, but you also have the option of defining a column in a DataFrame as an `index` and refer to rows using `index` labels instead of integer locations using `loc`.

For example in `rating_counts` defined above, the rating is the `index`, so we can reference a value either by its position or by its rating.

For example we can reference the 4th record either with `iloc` and its position or with `loc` and its label:

In [ ]:
rating_counts = None
rating_counts = ratings['rating'].value_counts()
print(rating_counts)
rating_counts.iloc[3]

In [ ]:
rating_counts.loc[3.5]

## Exercise 5: Set an Index for the Movies DataFrame

Movies have one column `movieId` that is a natural way of uniquely identifying each row, when that is the case, it is useful to turn that into an index.

In [ ]:
movies = movies.set_index("movieId")


In [ ]:
movies.head()

In [ ]:
ratings.head()

Ratings instead do not have a row identifier, both `userId` and `movieId` reference records in other dataframes, therefore there is no good candidate for an index so we can just leave the default integer indexing.

## Exercise 6: Year with Maximum Standard Deviation in the Rating

First assignment is to find which year has the maximum standard deviation in the rating, **not** the maximum value of the standard deviation, but the year when it occurs.
You can use the `idxmax` method, look at its documentation on the pandas website, https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.idxmax.html

First we want to convert the timestamp into a datetime object:

In [ ]:
# DO NOT MODIFY

ratings['parsed_time'] = pd.to_datetime(ratings['timestamp'], unit='s')

We can then access to datetime related fields through `dt`, for example:

In [ ]:
ratings.parsed_time.dt.month.head()

In [ ]:
ratings.rating.groupby(ratings.parsed_time.dt.year).std()

In [ ]:
ratings.rating.groupby(ratings.parsed_time.dt.year).std().idxmin()

In [ ]:
def find_year_with_max_std(ratings):
    """Function to find the year with the larger standard deviation in rating"""
    ### BEGIN SOLUTION
    return ratings.rating.groupby(ratings.parsed_time.dt.year).std().idxmin()
    ### END SOLUTION

In [ ]:
assert find_year_with_max_std(ratings) == 1995, "Wrong year identified, try again!"

## Advanced Exercise 1: Identify Popular Movies

*In the rest of the notebook, we will introduce new concepts not covered in class, this will challenge you to read additional pandas documentation.*

First we would like only to consider movies that have a significant number of ratings. This task is complicated by the fact that movies and ratings are in 2 different DataFrames and we want to filter the `movies` DataFrame based on a statistics on the `ratings` DataFrame.

First let's compute the number of ratings per movie:

In [ ]:
number_of_ratings = ratings.movieId.value_counts()

In [ ]:
number_of_ratings

Now we want to filter this pandas Series object and keep only the rows where the count is larger or equal to 100.
We don't want to pollute our analysis with movies with a tiny number of reviews:

In [ ]:
number_of_ratings_of_popular_movies = None
### BEGIN SOLUTION
number_of_ratings_of_popular_movies = number_of_ratings[number_of_ratings > 100]
print(number_of_ratings_of_popular_movies)
### END SOLUTION

In [ ]:
assert len(number_of_ratings_of_popular_movies) == 10326, "Try again, check that movies with 100 ratings are accepted"

Finally we want to use the `reindex` function to change the index of movies, this will create a new DataFrame with a new index that contains the movieId of only the most popular movies.

The value of all the rows of `movies` that have the same movieId will be copied over to the new `popular_movies` dataset, the rest will be discarded.

In [ ]:
all_popular_movies = movies.reindex(number_of_ratings_of_popular_movies.index)

In [ ]:
all_popular_movies.head(10)

## Advanced Exercise 2: Data Cleaning

Everytime we perform a reindexing operation, `pandas` will create a row for every value of the new index, even if it doesn't exist in the original data structure, and it will mark those data as missing with `NaN` (Not A Number).

Always check if reindexing generated invalid data:

In [ ]:
all_popular_movies.isnull().sum()

In the exercise above, we indexed the dataset with a list of matching values. Because of this our dataset does not have any `NaN`'s. This next exercise will be an example of what to do if this did happen! 

Go ahead and make a copy of the "number_of_ratings_of_popular_movies.index" from above and call it "fake_ratings_popular_movies_indices".

In [ ]:
fake_ratings_popular_movies_indices = None
### BEGIN SOLUTION

###END SOLUTION

Now if we try to modify these fake ratings we're going to run into a problem!! 

In [ ]:
fake_ratings_popular_movies_indices[0] = 2 #### The error is expected

Now we can see that certain sets in pandas are **Immutable**. This just means we can't make any changes to it, so we'll instead do things the wrong way for the rest of this exercise. (Just advanced exercise 2)

In [ ]:
### This is wrong code, just used for this example!
fake_ratings_popular_movies_indices = number_of_ratings_of_popular_movies
fake_popular_movies_ds = movies.reindex(fake_ratings_popular_movies_indices)
fake_popular_movies_ds.head()

In the next cell we want to drop the invalid data, look for a `pandas` function that performs that operation (it starts with "drop"!).

In [ ]:
popular_fake_movies = None
### BEGIN SOLUTION

### END SOLUTION

In [ ]:
assert len(popular_fake_movies) == 9435, "Try again, check the documentation of the function you used"

## Advanced Exercise 3: Filter by Genre

Okay now we're back to our real data! Let's implement a general function that filters movies by genres:

In [ ]:
def filter_by_genre(input_movies, genre):
    """Return only movies of a specific genre"""
    ### BEGIN SOLUTION

    ### END SOLUTION

Then let's apply it to the `popular_movies` dataset to retain only the "Fantasy" movies:

In [ ]:
fantasy_movies = filter_by_genre(all_popular_movies, "Fantasy")

In [ ]:
assert len(fantasy_movies) == 742, """Try again, Make sure you are filtering the popular movies"""

## Advanced Exercise 4: Join Movies and Ratings

Let's create a single `DataFrame` that contains both titles and mean ratings of the popular fantasy movies.

Titles are only available in the `movies` `DataFrame`, while ratings in the `ratings` `DataFrame`, we would like to create a single DataFrame that includes Title.

Create the `mean_ratings` variable by computing the mean rating for each movie:

In [ ]:
mean_ratings = None
### BEGIN SOLUTION

### END SOLUTION

In this case we don't even need to use a join operation, we can just create a new column in the `fantasy_movies` DataFrame. This will automatically match the index of `mean_ratings` with the index of `fantasy_movies` and attach to each movie its rating. Ratings for movies that are not in the `fantasy_movies` DataFrame are discarded.

The recommended way of creating columns in the recent versions of `pandas` is through the `assign` function, read its documentation!

In [ ]:
fantasy_movies.assign?

In [ ]:
fantasy_movies_with_ratings = fantasy_movies.assign(rating = mean_ratings)

In [ ]:
fantasy_movies_with_ratings.head()

In [ ]:
assert fantasy_movies_with_ratings.loc[7164].title.startswith("Peter"), "Try again, missing or wrong title"

In [ ]:
assert abs(fantasy_movies_with_ratings.loc[7164].rating - 3.39) < 0.01 , "Try again, missing or wrong rating"

## Advanced Exercise 5: Find the Highest Rated Fantasy Movie

Again we need to find the index where a column is max, in this case rating:

In [ ]:
index_of_max_rating = None
### BEGIN SOLUTION

### END SOLUTION

In [ ]:
highest_rated_fantasy_movie = fantasy_movies_with_ratings.loc[index_of_max_rating]

In [ ]:
assert highest_rated_fantasy_movie.title.startswith("Spirited"), "Try again"